<a href="https://colab.research.google.com/github/PietroVolpato/lfn_project/blob/main/src/LFN_project_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning from networks project
### Evaluation of different Node Embedding algorithms
Members:<br>
- D'Emilio Filippo, id : 2120931
- Volpato Pietro, id : 2120825

### Information about the notebook (have a look at the report for details)
This notebook is responsable of computing the embeddings for every embedding technique and for every selected graph.<br>
Each computed embedding is saved to file as a numpy array (extension .npy), in the directory /embeddings. In this way that once an embedding is computed, it won't be lost when the runtime of the notebook is terminated.<br>
We can then efficiently load the embeddings in the "test" notebook, and evaluate the quality of the embeddings.<br>
Selected embedding techniques:
- Node2Vec
- Line
- ...

For information about the graphs, se cells below.<br>
*NOTE*: by implementation choice, the computation of each embedding is computed separately (e.g. there are no function to coincisely compute all embeddings).<br>
This choice comes from the fact that computing embeddings is computationally intensive, and we might want to compute only a specific
embedding strategy for a specific graph, in order to update only this entry in the folder containing the embeddings.

### Imports

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from node2vec import Node2Vec
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import gzip
import sys
import re

c:\Users\pietr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# configuration
Here you can properly configure the names of the graphs and the names of the embedding strategies. Use meaningful names.

In [2]:
graph_keys = ["facebook","citation","biological","CL","COX2"]
embedding_keys = ["LINE", "node2vec"]

# Loading the graphs
Selected graphs:
- Facebook_combined    https://snap.stanford.edu/data/ego-Facebook.html          
- cit-Helpth           https://networkrepository.com/cit-HepTh.php             
- bio-CE-CX            https://networkrepository.com/bio-CE-CX.php             
- CL-100K-1d8-L9       https://networkrepository.com/CL-100K-1d8-L9.php ---- the graph has node labels
- COX2-MD              https://networkrepository.com/COX2-MD.php  ---- the graph has node labels

To run this notebook, adjust the paths to match where the files are saved in your PC.<br>
To keep paths as they are, create a "data" folder inside the directory of this notebook, and store the files there.<br><br>

Graphs are stored as a dictionary: the key is the graph name, the value is the corresponding netowrkx graph.<br>

In [ ]:
facebook_path = 'data/facebook_combined.txt.gz'
citation_path = 'data/cit-HepTh.edges'
biological_path = 'data/bio-CE-CX.edges'
CL_path = "data/CL-100K-1d8-L9/CL-100K-1d8-L9.edges"
COX2_path = "data/COX2-MD/COX2-MD.edges"

In [30]:
def load_graph(path):
    """
    For files with extension .edges
    """
    G = nx.Graph()
    with open(path, 'rt') as f:
        for line in f:
            if line.startswith('%'):  # Skip comment lines
                continue
            # Split the line based on spaces or commas
            data = re.split(r'[,\s]+', line.strip())
            if len(data) < 2:  # Skip lines that don't have at least two columns
                continue
            # Extract the first two columns (nodes)
            node1, node2 = int(data[0]), int(data[1])
            G.add_edge(node1, node2)
    mapping = {node : i for i,node in enumerate(G.nodes)} # mappoing original : relabeled
    G = nx.relabel_nodes(G, mapping)
    return G

def load_graph_with_gz(path):
    """
    For files with extension .txt.gz
    """
    G = nx.Graph()
    with gzip.open(path, 'rt') as f:
        for line in f:
            node1, node2 = map(int, line.strip().split())
            G.add_edge(node1, node2)
    mapping = {node : i for i,node in enumerate(G.nodes)} # mappoing original : relabeled
    G = nx.relabel_nodes(G, mapping)
    return G

def print_graphs_info(graphs):
    for k in graph_keys:
        G = graphs[k]
        print(f"{k}: |V|={len(G.nodes)}, |E|={len(G.edges)}")

In [31]:
graphs = {}

# facebook graph is the only one .tar.gz
graphs[graph_keys[0]] = load_graph_with_gz(facebook_path)  # relabeling nodes to integer
graphs[graph_keys[1]] = load_graph(citation_path)
graphs[graph_keys[2]] = load_graph(biological_path)
graphs[graph_keys[3]] = load_graph(CL_path)  # node labeled
graphs[graph_keys[4]] = load_graph(COX2_path)  # node labeled

print_graphs_info(graphs)

facebook graph: |V|=4039, |E|=88234
citation graph: |V|=22908, |E|=2444798
biological graph: |V|=15229, |E|=245952
CL graph: |V|=92482, |E|=436611
COX2 graph: |V|=7962, |E|=101542


# Download the dataset from the GitHub repository

In [9]:
import requests

url = "https://raw.githubusercontent.com/PietroVolpato/lfn_project/main/data/"
filename = "bio-CE-CX_edges.csv"

response = requests.get(url + filename)
with open(filename, "wb") as file:
    file.write(response.content)

# Functions and declarations for the embeddings
Embedding data structure is defined as following:<br>
- The first index refer to the graph (e.g. embeddings["facebook"] contains the embeddings of the facebook graph for every embedding technique).<br>
- The second index refer to the embedding technique (e.g. embeddings["facebook"]["LINE"] cointans the embedding of facebook graph computed using LINE)

In [21]:
def save(emb, graph_key, embedding_key):
    path = f"../result/embeddings_{graph_key}_{embedding_key}.npy"
    np.save(path, emb)
    print(f"Successfully saved the embeddings in {path}")

# dictionaries to store the embeddings, obtained by several techniques, for each graph
embeddings = {}
for k in graph_keys:
    embeddings[k] = {}

Spiegazione sui parametri di node2vec:<br>
- G (required): The graph on which to run Node2Vec. Must be an undirected networkx.Graph object.
- dimensions (default = 128): The dimensionality of the node embeddings. Higher dimensions allow for capturing more information but increase computational cost.
- walk_length (default = 80): The number of steps for each random walk. A larger walk_length captures more of the network structure.
- num_walks (default = 10): The number of random walks to start per node. Increasing this can improve the representation at the cost of additional computation.
- workers (default = 1): The number of CPU cores to use for parallel processing. If you're running this on a multi-core machine, increasing this can speed up the computation.
- p (return parameter): p<1: Increases the likelihood of revisiting a node (DFS-like behavior). p>1: Discourages revisiting nodes, encouraging exploration (BFS-like behavior).
- q (in-out parameter): q<1: Encourages walks to nodes further away from the starting node (BFS-like).q>1: Biases walks to nodes closer to the starting node (DFS-like).

Spiegazione di : model = node2vec.fit(window=5, min_count=1, batch_words=4)<br>
This trains a Word2Vec model (from the gensim library) using the random walks. Let’s go over the parameters:<br>

- window (default = 10): The maximum distance between the current and predicted nodes in the random walk sequence. Larger windows capture more context but require more computation.

- min_count (default = 1): Minimum frequency for a node to be considered in the embedding. Since most graphs are sparse, this is often set to 1.

- batch_words (default = 4): The number of words (or nodes) processed in each training batch. Adjust this for performance depending on your hardware.

# Node2Vec
- pip install node2vec

In [10]:
def get_node2vec_embeddings(G, dimensions=128, walk_length=10, num_walks=20, p=1, q=1, workers=1):
    """
    Generate node embeddings for a graph using the Node2Vec algorithm.

    Parameters:
        G (networkx.Graph):
            The input graph for which embeddings are to be generated.
            The graph should have nodes labeled as integers, ideally sequentially starting from 0.

        dimensions (int, optional):
            The dimensionality of the embedding space. Default is 128.

        walk_length (int, optional):
            The length of each random walk. Default is 10.

        num_walks (int, optional):
            The number of random walks to start from each node. Default is 20.

        p (float, optional):
            The return parameter, controlling the likelihood of immediately revisiting a node in the walk.
            A higher value makes it more likely to backtrack. Default is 1.

        q (float, optional):
            The in-out parameter, controlling the likelihood of exploring outward from the starting node.
            A higher value makes it more likely to move outward. Default is 1.

        workers (int, optional):
            The number of parallel workers for random walk generation and model training. Default is 1.

    Returns:
        np.ndarray:
            A NumPy array where each row represents the embedding of a node.
            The row index corresponds to the node ID, and each row has `dimensions` elements.
    """
    # Initialize Node2Vec model
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, p=p, q=q, workers=workers)

    # Fit the Node2Vec model and generate embeddings
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # Convert embeddings to a NumPy array
    num_nodes = G.number_of_nodes()
    embeddings = np.zeros((num_nodes, dimensions))  # Preallocate array
    for node in G.nodes:
        embeddings[node] = model.wv[node]

    return embeddings

## Produce the embeddings with node2vec
here you can easily produce the embeddings for any of the loaded graphs using node2vec.<br>
Adjust the variable curr_graph_key with the key of the graph you want to compute the embeddings for.<br>
The embeddings are saved to file (look output to get path).

In [23]:
# graph_keys[0] = facebook
# graph_keys[1] = biological
# graph_keys[2] = citation
# graph_keys[3] = CL
# graph_keys[4] = COX2
curr_graph_key = graph_keys[2]   # chose the graph

embeddings[curr_graph_key]["node2vec"] = get_node2vec_embeddings(graphs[curr_graph_key])
save(embeddings[curr_graph_key]["node2vec"], curr_graph_key, "node2vec")

Computing transition probabilities:   0%|          | 0/15229 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 20/20 [00:31<00:00,  1.56s/it]


Successfully saved the embeddings in ../result/embeddings_biological_node2vec.npy


# LINE : Large-scale information network embedding
installation guide:
- git clone https://github.com/VahidooX/LINE.git
- !pip install keras
- !pip install tensorflow
- adjust the sys.path to where you downloaded LINE repository

*NOTE*: it was necessary to modify utils.py to adapt it at current version of keras because some elements were deprecated

In [24]:
sys.path.append(r'C:\Users\oppil\OneDrive\Desktop\Universita\magistrale\2_1\LFN\REPO_PROJECT\lfn_project\src\LINE')

from model import create_model
from utils import batchgen_train

def get_LINE_embeddings(G, embedding_dim=128, batch_size=1024, negative_ratio=5, epochs=10, negative_sampling="UNIFORM"):
    """
    Generate LINE embeddings for a given graph.

    Parameters:
        G (nx.Graph): The graph for which embeddings are computed.
        embedding_dim (int): Dimensionality of the embeddings.
        batch_size (int): Batch size for training.
        negative_ratio (int): Ratio of negative to positive samples.
        epochs (int): Number of training epochs.
        negative_sampling (str): Negative sampling strategy ("UNIFORM" or "NON-UNIFORM").

    Returns:
        numpy.ndarray: Node embeddings (shape: [num_nodes, embedding_dim]).
    """
    num_nodes = G.number_of_nodes()

    # Convert networkx.Graph to adj_list (edge list as 2D numpy array)
    adj_list = np.array(list(G.edges()), dtype=np.int32)

    # Create LINE model
    model, embed_generator = create_model(num_nodes, embedding_dim)

    # Generate training batches
    train_gen = batchgen_train(adj_list, num_nodes, batch_size, negative_ratio, negative_sampling)

    # Compile and train the model
    model.compile(optimizer="adam", loss="binary_crossentropy")
    model.fit(train_gen, steps_per_epoch=500, epochs=epochs)

    # Extract embeddings
    node_ids = np.arange(num_nodes)  # Sequential node IDs
    embeddings = embed_generator.predict_on_batch(node_ids)

    print("Node Embeddings Shape:", embeddings[0].shape)
    return embeddings

## Produce the embeddings with LINE
here you can easily produce the embeddings for any of the loaded graphs using LINE.<br>
Adjust the variable curr_graph_key with the key of the graph you want to compute the embeddings for.<br>
The embeddings are saved to file (look output to get path).

In [26]:
curr_graph_key = graph_keys[0]   # chose the graph

embeddings[curr_graph_key]["LINE"] = get_LINE_embeddings(graphs[curr_graph_key], epochs = 20)
save(embeddings[curr_graph_key]["LINE"], curr_graph_key, "LINE")

Epoch 1/20
500/500 [==============================] - 57s 113ms/step - loss: 0.9791
Epoch 2/20
500/500 [==============================] - 59s 117ms/step - loss: 0.4258
Epoch 3/20
500/500 [==============================] - 59s 118ms/step - loss: 0.3137
Epoch 4/20
500/500 [==============================] - 57s 114ms/step - loss: 0.2445
Epoch 5/20
500/500 [==============================] - 58s 117ms/step - loss: 0.2016
Epoch 6/20
500/500 [==============================] - 58s 116ms/step - loss: 0.1660
Epoch 7/20
500/500 [==============================] - 58s 116ms/step - loss: 0.1444
Epoch 8/20
500/500 [==============================] - 60s 119ms/step - loss: 0.1291
Epoch 9/20
500/500 [==============================] - 57s 114ms/step - loss: 0.1187
Epoch 10/20
500/500 [==============================] - 61s 123ms/step - loss: 0.1092
Epoch 11/20
500/500 [==============================] - 69s 137ms/step - loss: 0.1037
Epoch 12/20
500/500 [==============================] - 58s 116ms/step - lo

# AttentionWalk

## Installation guide
<ol>
<li>git clone https://github.com/benedekrozemberczki/AttentionWalk.git</li>
<li>pip install texttable</li>
</ol>

It requires that the input file is a .csv, so first we have implemented a function that converts the .txt.gz and the .edges files to a .csv to be given as input to the AttentionWalk algorithm.<br>
For starting the algorithm you have to enter to the AttentionWalk folder after having cloned it from the Github repository and then set the arguments as described in the README.md file.

In [ ]:
!git clone https://github.com/benedekrozemberczki/AttentionWalk.git

In [ ]:
!pip install texttable

## Test with the facebook network
Save the embeddings in the result folder with also the attention path<br>
Time: 1m 50s

In [1]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/facebook_combined.csv --embedding-path ../../result/facebook_embeddings_attention.csv --attention-path ../../result/facebook_attention.csv

+----------------+------------------------------------------------+
| Attention path |      ../../result/facebook_attention.csv       |
+================+================================================+
| Beta           | 0.500                                          |
+----------------+------------------------------------------------+
| Dimensions     | 128                                            |
+----------------+------------------------------------------------+
| Edge path      | ../../data/facebook_combined.csv               |
+----------------+------------------------------------------------+
| Embedding path | ../../result/facebook_embeddings_attention.csv |
+----------------+------------------------------------------------+
| Epochs         | 200                                            |
+----------------+------------------------------------------------+
| Gamma          | 0.500                                          |
+----------------+------------------------------


Adjacency matrix powers: 100%|██████████| 5/5 [00:02<00:00,  2.11it/s]

Training Progress: 100%|██████████| 200/200 [04:03<00:00,  1.22s/it]


## Test with the citation network
Infeasible!!

In [ ]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/cit-HepTh_edges.csv --embedding-path ../../result/cit-HepTh_embeddings_attention.csv --attention-path ../../result/cit-HepTh_attention.csv --epochs 176

## Test with the biological network
Infeasible!!!

In [ ]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/bio-CE-CX_edges.csv --embedding-path ../../result/bio-CE-CX_embeddings_attention.csv --attention-path ....//result/bio-CE-CX_attention.csv

## Test with email-Enron
Infeasibile

In [ ]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/email-Enron.csv --embedding-path ../../result/email-Enron_embeddings_attention.csv --attention-path ../../result/email-Enron_attention.csv

## Test with CL-100K-1d8-L9

In [1]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/CL-100K-1d8-L9.csv --embedding-path ../../result/CL-100K-1d8-L9_embeddings_attention.csv --attention-path ../../result/CL-100K-1d8-L9_attention.csv

+----------------+------------------------------------------------------+
| Attention path |      ../../result/CL-100K-1d8-L9_attention.csv       |
+================+======================================================+
| Beta           | 0.500                                                |
+----------------+------------------------------------------------------+
| Dimensions     | 128                                                  |
+----------------+------------------------------------------------------+
| Edge path      | ../../data/CL-100K-1d8-L9.csv                        |
+----------------+------------------------------------------------------+
| Embedding path | ../../result/CL-100K-1d8-L9_embeddings_attention.csv |
+----------------+------------------------------------------------------+
| Epochs         | 200                                                  |
+----------------+------------------------------------------------------+
| Gamma          | 0.500              


Adjacency matrix powers:  60%|██████    | 3/5 [01:46<01:11, 35.56s/it]
Traceback (most recent call last):
  File "c:\Users\pietr\OneDrive\Desktop\lfn_project\src\AttentionWalk\src\main.py", line 19, in <module>
    main()
  File "c:\Users\pietr\OneDrive\Desktop\lfn_project\src\AttentionWalk\src\main.py", line 14, in main
    model = AttentionWalkTrainer(args)
  File "c:\Users\pietr\OneDrive\Desktop\lfn_project\src\AttentionWalk\src\attentionwalk.py", line 71, in __init__
    self._initialize_model_and_data()
  File "c:\Users\pietr\OneDrive\Desktop\lfn_project\src\AttentionWalk\src\attentionwalk.py", line 75, in _initialize_model_and_data
    sparse_target_tensor = feature_calculator(self.args, self.graph)
  File "c:\Users\pietr\OneDrive\Desktop\lfn_project\src\AttentionWalk\src\utils.py", line 56, in feature_calculator
    powered_A = powered_A @ normalized_adjacency_matrix
  File "c:\Users\pietr\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_base.py", line 69

## Test with COX2-MD

In [2]:
import pandas as pd
import networkx as nx

def load_graph_from_edgelist(path):
    """
    Load a graph from an edge list file.

    Parameters:
        path (str): Path to the edge list file.

    Returns:
        nx.Graph: The loaded graph.
    """
    df = pd.read_csv(path, header=None)
    G = nx.from_pandas_edgelist(df, source=0, target=1)
    return G

G = load_graph_from_edgelist("../data/COX2-MD.csv")
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

# Check if the graph is connected
print("Is the graph connected?", nx.is_connected(G))
if not nx.is_connected(G):
    print("Number of connected components:", nx.number_connected_components(G))

Number of nodes: 7964
Number of edges: 101543
Is the graph connected? False
Number of connected components: 304


In [2]:
!cd AttentionWalk && python src/main.py --edge-path ../../data/COX2-MD.csv --embedding-path ../../result/COX2-MD_embeddings_attention.csv --attention-path ../../result/COX2-MD_attention.csv

+----------------+-----------------------------------------------+
| Attention path |      ../../result/COX2-MD_attention.csv       |
+================+===============================================+
| Beta           | 0.500                                         |
+----------------+-----------------------------------------------+
| Dimensions     | 128                                           |
+----------------+-----------------------------------------------+
| Edge path      | ../../data/COX2-MD.csv                        |
+----------------+-----------------------------------------------+
| Embedding path | ../../result/COX2-MD_embeddings_attention.csv |
+----------------+-----------------------------------------------+
| Epochs         | 200                                           |
+----------------+-----------------------------------------------+
| Gamma          | 0.500                                         |
+----------------+--------------------------------------------


Adjacency matrix powers: 100%|██████████| 5/5 [00:00<00:00, 138.89it/s]

Training Progress: 100%|██████████| 200/200 [09:21<00:00,  2.81s/it]


# GAE

In [ ]:
!cd gae && python setup.py install

In [ ]:
!cd gae/gae && python train.py